# Library imports

In [ ]:
import cv2
import pickle
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

import tensorflow as tf
sess = tf.Session()

from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
K.set_session(sess)

from keras.models import Sequential, Model
from keras.layers import Activation, Input, Embedding, LSTM, Dense, merge, Convolution2D, MaxPooling2D, Reshape, Flatten, Dropout
from keras.optimizers import RMSprop, Adam
from keras import initializations

# Data loading

In [ ]:
training_file = '../train.p'
testing_file = '../test.p'


with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)


X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

# Data normalization

In [3]:
# normalize data
X_train = X_train.astype(np.float32)/255
X_train -= 0.5

y_label = np.zeros(shape=(y_train.shape[0], np.max(y_train) + 1))

for i in range(y_train.shape[0]):
    y_label[i, y_train[i]]=1

# Data augmentation

In [4]:
class LavoeImageDataGenerator(ImageDataGenerator):
 
    def random_transform(self, x):
        # ImageGenerator from Keras was providing negative images, so I nverted the original input
        img = 256 - x
        
        ang_range = 2
        shear_range = 2
        trans_range = 2
        brightness = False

        # Rotation
        ang_rot = np.random.uniform(ang_range)-ang_range/2
        rows,cols,ch = img.shape    
        Rot_M = cv2.getRotationMatrix2D((cols/2,rows/2),ang_rot,1)

        # Translation
        tr_x = trans_range*np.random.uniform()-trans_range/2
        tr_y = trans_range*np.random.uniform()-trans_range/2
        Trans_M = np.float32([[1,0,tr_x],[0,1,tr_y]])

        # Shear
        pts1 = np.float32([[5,5],[20,5],[5,20]])

        pt1 = 5+shear_range*np.random.uniform()-shear_range/2
        pt2 = 20+shear_range*np.random.uniform()-shear_range/2

        # Brightness 
        pts2 = np.float32([[pt1,5],[pt2,pt1],[5,pt2]])

        shear_M = cv2.getAffineTransform(pts1,pts2)

        img = cv2.warpAffine(img,Rot_M,(cols,rows))
        img = cv2.warpAffine(img,Trans_M,(cols,rows))
        img = cv2.warpAffine(img,shear_M,(cols,rows))


        if brightness:
            img = cv2.cvtColor(img,cv2.COLOR_RGB2HSV)
            random_bright = .25+np.random.uniform()
            #print(random_bright)
            img[:,:,2] = img[:,:,2]*random_bright
            img = cv2.cvtColor(img,cv2.COLOR_HSV2RGB)

        # Make sure the output is from 0 - 255.
        img = img.astype(np.uint8)
        return img

# Model saving setup and initialization

In [5]:
def my_init(shape, name=None):
    return initializations.normal(shape, scale=0.01, name=name)

# Network Architecture

In [6]:
# input layer
main_input = Input(shape=(32, 32, 3), name='main_input')

color_map = Convolution2D(3, 1, 1, init=my_init, border_mode='same')(main_input)

conv1 = Convolution2D(32, 3, 3, init=my_init, border_mode='same')(color_map)
conv2 = Convolution2D(32, 3, 3, init=my_init, border_mode='same')(conv1)
pool1 = MaxPooling2D(pool_size=(3, 3))(conv2)
drop1 = Dropout(0.5)(pool1)

conv3 = Convolution2D(64, 3, 3, init=my_init, border_mode='same')(drop1)
conv4 = Convolution2D(64, 3, 3, init=my_init, border_mode='same')(conv3)
pool2 = MaxPooling2D(pool_size=(3, 3))(conv4)
drop2 = Dropout(0.5)(pool2)

conv5 = Convolution2D(128, 3, 3, init=my_init, border_mode='same')(drop2)
conv6 = Convolution2D(128, 3, 3, init=my_init, border_mode='same')(conv5)
pool3 = MaxPooling2D(pool_size=(3, 3))(conv6)
drop3 = Dropout(0.5)(pool3)

# flatten layer
flat1 = Flatten()(drop1)
flat2 = Flatten()(drop2)
flat3 = Flatten()(drop3)

# concatenate all vectors
merged1 = merge([flat1, flat2, flat3], name='merge1', mode='concat',concat_axis = 1)

fc1 = Dense(1024)(merged1)
drop4 = Dropout(0.5)(fc1)

fc2 = Dense(1024)(drop4)
drop5 = Dropout(0.5)(fc2)

softmax_layer = Dense(43)(drop5)
main_output = Activation('softmax')(softmax_layer)
print("final", main_output)

final Tensor("Softmax:0", shape=(?, 43), dtype=float32)


In [7]:
datagen = LavoeImageDataGenerator(width_shift_range=0.2,
                             height_shift_range=0.2,
                             rotation_range=20,
                             zoom_range=0.2,
                             shear_range=0,
                            )

# Multi GPU


In [ ]:
from keras.layers import merge
from keras.layers.core import Lambda
from keras.models import Model

import tensorflow as tf

def make_parallel(model, gpu_count):
    def get_slice(data, idx, parts):
        shape = tf.shape(data)
        size = tf.concat(0, [ shape[:1] // parts, shape[1:] ])
        stride = tf.concat(0, [ shape[:1] // parts, shape[1:]*0 ])
        start = stride * idx
        return tf.slice(data, start, size)

    outputs_all = []
    for i in range(len(model.outputs)):
        outputs_all.append([])

    #Place a copy of the model on each GPU, each getting a slice of the batch
    for i in range(gpu_count):
        with tf.device('/gpu:%d' % i):
            with tf.name_scope('tower_%d' % i) as scope:

                inputs = []
                #Slice each input into a piece for processing on this GPU
                for x in model.inputs:
                    input_shape = tuple(x.get_shape().as_list())[1:]
                    slice_n = Lambda(get_slice, output_shape=input_shape, arguments={'idx':i,'parts':gpu_count})(x)
                    inputs.append(slice_n)                

                outputs = model(inputs)
                
                if not isinstance(outputs, list):
                    outputs = [outputs]
                
                #Save all the outputs for merging back together later
                for l in range(len(outputs)):
                    outputs_all[l].append(outputs[l])

    # merge outputs on CPU
    with tf.device('/cpu:0'):
        merged = []
        for outputs in outputs_all:
            merged.append(merge(outputs, mode='concat', concat_axis=0))
            
        return Model(input=model.inputs, output=merged)

In [ ]:
gpu_count = 16
batch_size = 2048
prefix = 'traffic'
model_path ='.model_{0}.h5'.format(prefix)
epochs = 200
learning_rate = 0.1
decay_rate = learning_rate / epochs
sgd_select = Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=decay_rate)

dataset_size = gpu_count * batch_size

sign_model = Model(input = [main_input], output = [main_output])

sign_model = make_parallel(sign_model, gpu_count)

sign_model.compile(optimizer = sgd_select, loss = 'categorical_crossentropy', metrics=['accuracy'])

sign_model.fit_generator(datagen.flow(X_train[:dataset_size], y_label[:dataset_size], shuffle=True, batch_size=gpu_count * batch_size),
                    samples_per_epoch=dataset_size, nb_epoch=epochs)
sign_model.save(model_path)
K.clear_session()

Epoch 1/2000
32768/32768 [==============================] - 27s - loss: 3.7734 - acc: 0.0230
Epoch 2/2000
32768/32768 [==============================] - 1s - loss: 3.7364 - acc: 0.0394
Epoch 3/2000
32768/32768 [==============================] - 1s - loss: 3.6970 - acc: 0.0598
Epoch 4/2000
32768/32768 [==============================] - 1s - loss: 3.6521 - acc: 0.0733
Epoch 5/2000
32768/32768 [==============================] - 1s - loss: 3.6032 - acc: 0.0756
Epoch 6/2000
32768/32768 [==============================] - 1s - loss: 3.5495 - acc: 0.0800
Epoch 7/2000
32768/32768 [==============================] - 1s - loss: 3.4983 - acc: 0.0776
Epoch 8/2000
32768/32768 [==============================] - 1s - loss: 3.4550 - acc: 0.0800
Epoch 9/2000
32768/32768 [==============================] - 1s - loss: 3.4219 - acc: 0.0790
Epoch 10/2000
32768/32768 [==============================] - 3s - loss: 3.4040 - acc: 0.0833
Epoch 11/2000
32768/32768 [==============================] - 3s - loss: 3.3923